In [4]:
import pandas as pd
import mat4py as mp
from glob import glob
from datetime import datetime

pd.set_option('io.hdf.default_format','table')

In [5]:
files = glob('data/0-raw/*.mat')

In [6]:
def date_from_filename(filename):
    parsed = filename.split('_')
    day = int(parsed[1])
    month = int(parsed[2])
    year = int(parsed[3])
    hour = int(parsed[4])
    minute = int(parsed[5][:2])
    return datetime(year, month, day, hour, minute)

In [82]:
for file in files:
    date = date_from_filename(file)
    data = pd.DataFrame(mp.loadmat(file))
    data.index = data.index.set_names('Trial')
    data['datetime'] = date
    
    eye_samples = data.explode('online_eye')
    eye_samples[['Pupils','x','y','t']] = pd.DataFrame(eye_samples['online_eye'].tolist(), index=eye_samples.index)
    eye_samples = eye_samples.reset_index()
    eye_samples = eye_samples.drop(columns=['online_eye'])
    
    trials = data.drop(columns=['online_eye'])
    trials = trials.set_index(['datetime',trials.index])
    trial_codes = {6101:'Novel', 6111: 'Novel Learning', 6102: 'Familiar', 6112: 'Familiar Learning'}
    trials['type'] = trials['trial_type'].map(trial_codes)
    trials['t0'] = eye_samples.groupby(['datetime','Trial']).first()['t']
    trials_eye = trials.reset_index().merge(eye_samples)
    eye_samples['timedelta'] = trials_eye['t'] - trials_eye['t0']
    eye_samples.to_hdf('data/1-normalized.hdf','eye_samples', append=True)
    trials.to_hdf('data/1-normalized.hdf','trials', append=True)

data/0-raw/BFnovelinac_01_02_2019_15_03.mat
data/0-raw/BFnovelinac_01_02_2019_16_08.mat
data/0-raw/BFnovelinac_03_12_2018_12_52.mat
data/0-raw/BFnovelinac_03_12_2018_13_15.mat
data/0-raw/BFnovelinac_05_12_2018_13_06.mat
data/0-raw/BFnovelinac_07_12_2018_09_21.mat
data/0-raw/BFnovelinac_07_12_2018_10_10.mat
data/0-raw/BFnovelinac_07_12_2018_11_15.mat
data/0-raw/BFnovelinac_09_11_2018_15_21.mat
data/0-raw/BFnovelinac_12_11_2018_16_19.mat
data/0-raw/BFnovelinac_12_11_2018_16_25.mat
data/0-raw/BFnovelinac_12_11_2018_16_31.mat
data/0-raw/BFnovelinac_13_11_2018_15_01.mat
data/0-raw/BFnovelinac_13_12_2018_10_54.mat
data/0-raw/BFnovelinac_13_12_2018_11_02.mat
data/0-raw/BFnovelinac_13_12_2018_11_44.mat
data/0-raw/BFnovelinac_13_12_2018_12_57.mat
data/0-raw/BFnovelinac_14_01_2019_14_44.mat
data/0-raw/BFnovelinac_14_01_2019_16_21.mat
data/0-raw/BFnovelinac_14_11_2018_15_35.mat
data/0-raw/BFnovelinac_14_11_2018_16_09.mat
data/0-raw/BFnovelinac_15_01_2019_14_28.mat
data/0-raw/BFnovelinac_15_01_201

In [23]:
trials = pd.read_hdf('data/1-normalized.hdf','trials')
eye_samples = pd.read_hdf('data/1-normalized.hdf','eye_samples')